In [81]:
def getfile(filename, dict_cat2int, count):
    a = 0
    dict = {}
    with open(filename, "r", encoding = "ISO-8859-1") as f:
        line = f.readline()
        while line != None and line != "" and a < count:
            arr = line.split("\t")[0][:-1].split('::')
            index = arr[0]
            value = arr[2].split('|')
            cat_int = [dict_cat2int[val] for val in value]
            dict[index] = [0 + 1 * (i in cat_int)for i in range(18)]
            a += 1
            line = f.readline()
    return dict

In [75]:
def get_dict_cat2int(filename):
    a = 0
    dict = {}
    with open(filename, "r", encoding = "ISO-8859-1") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")[0][:-1].split('::')
            index = arr[0]
            value = arr[2].split('|')
            for val in value:
                if val not in dict:
                    dict[val] = a
                    a += 1
            if a == 18:
                break
            line = f.readline()
    return dict

In [83]:
import pickle
def save_object(obj, filename):
  with open(filename, 'wb') as output:  # Overwrites any existing file.
    pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [68]:
movies = '/Users/hp/GitHub/neural_collaborative_filtering/Data/movies.dat'

In [71]:
dict_cat2int = get_dict_cat2int(movies)
dict_cat2int

{'Action': 7,
 'Adventure': 3,
 'Animation': 0,
 "Children's": 1,
 'Comedy': 2,
 'Crime': 8,
 'Documentary': 12,
 'Drama': 6,
 'Fantasy': 4,
 'Film-Noir': 16,
 'Horror': 10,
 'Musical': 14,
 'Mystery': 15,
 'Romance': 5,
 'Sci-Fi': 11,
 'Thriller': 9,
 'War': 13,
 'Western': 17}

In [82]:
dict_id2cat = getfile(movies, dict_cat2int, count = 100)
save_object(dict_id2cat, ./data/)

In [33]:
getfile('/Users/hp/GitHub/neural_collaborative_filtering/Data/ml-1m.train.rating',10)

['0', '32', '4', '978824330\n']
['0', '34', '4', '978824330\n']
['0', '4', '5', '978824291\n']
['0', '35', '4', '978824291\n']
['0', '30', '4', '978824291\n']
['0', '29', '3', '978824268\n']
['0', '33', '4', '978824268\n']
['0', '40', '5', '978824268\n']
['0', '10', '5', '978824268\n']
['0', '16', '3', '978824268\n']


In [34]:
getfile('/Users/hp/GitHub/neural_collaborative_filtering/Data/movies.dat',10)

["1::Toy Story (1995)::Animation|Children's|Comedy\n"]
["2::Jumanji (1995)::Adventure|Children's|Fantasy\n"]
['3::Grumpier Old Men (1995)::Comedy|Romance\n']
['4::Waiting to Exhale (1995)::Comedy|Drama\n']
['5::Father of the Bride Part II (1995)::Comedy\n']
['6::Heat (1995)::Action|Crime|Thriller\n']
['7::Sabrina (1995)::Comedy|Romance\n']
["8::Tom and Huck (1995)::Adventure|Children's\n"]
['9::Sudden Death (1995)::Action\n']
['10::GoldenEye (1995)::Action|Adventure|Thriller\n']


In [ ]:
'''
Created on Aug 8, 2016
Processing datasets. 

@author: Xiangnan He (xiangnanhe@gmail.com)
'''
import scipy.sparse as sp
import numpy as np

class Dataset(object):
    '''
    classdocs
    '''

    def __init__(self, path):
        '''
        Constructor
        '''
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape
        
    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList
    
    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        '''
        Read .rating file and Return dok matrix.
        The first line of .rating file is: num_users\t num_items
        '''
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        # Construct matrix
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()    
        return mat
